<a href="https://colab.research.google.com/github/mattprodani/talentless-artist/blob/main/ArtistGAN_ForAll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PaintingGAN**

Build a Machine Learning model to generate paintings based on your favorite artist.

Tutorial provided by Matt Prodani - mp5908@nyu.edu

Many thanks to the resources provided by NVidia, Ikarus777, and the artists that made the paintings.

This Colab is published non-comercially for research under the same license as StyleGAN2 available at:
https://nvlabs.github.io/stylegan2-ada-pytorch/license.html

**First run:** Go through entire Colab adjusting as needed.
**Continuing runs** Install dependencies, Skip accessing and preprocessing, Mount Google Drive, and continue to the section on continuing training.

Basic Dependencies

In [ ]:
!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

# Accessing and processing data:
For this step, you can upload through the files menu on the left side, or through Colab. Make note of where your files are located.

This entire section only needs to be run once, at setup. The data will be saved on Google Drive for later use. You can rerun it if you want to download a new dataset for training.

Downloading the Kaggle dataset

For this you will need a "kaggle.json" file which you can download by logging into kaggle and creating an authentication key on https://www.kaggle.com/me/account. 
You can then upload it through the folder on the left toolbar

In [ ]:
# Skip this cell if you are using a custom dataset

# Upload kaggle.json through the files tab to the content folder.
# - https://www.kaggle.com/me/account - Navigate to authentication, create a key and download kaggle.json


!pip install kaggle
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ikarus777/best-artworks-of-all-time
!unzip best-artworks-of-all-time



In [ ]:
#Clean up unnecessary data
!rm -r /content/resized
!rm /content/artists.csv

The code below is specific to the structure of this dataset, to use on other datasets, follow the logic from inside the inner for loop. Then continue to the setting up StyleGAN section.

Change DIMS to your preffered size, larger sizes will provide higher quality results but will take much longer. Square images are preferred.

Use this to see the artists available in the dataset

In [ ]:
%cd /content/images/images 
!ls

/content/images/images
Albrecht_Du╠Иrer     Frida_Kahlo		Pablo_Picasso
Albrecht_DuтХа├кrer  Georges_Seurat		Paul_Cezanne
Alfred_Sisley	     Giotto_di_Bondone		Paul_Gauguin
Amedeo_Modigliani    Gustave_Courbet		Paul_Klee
Andrei_Rublev	     Gustav_Klimt		Peter_Paul_Rubens
Andy_Warhol	     Henri_de_Toulouse-Lautrec	Pierre-Auguste_Renoir
Camille_Pissarro     Henri_Matisse		Pieter_Bruegel
Caravaggio	     Henri_Rousseau		Piet_Mondrian
Claude_Monet	     Hieronymus_Bosch		Raphael
Diego_Rivera	     Jackson_Pollock		Rembrandt
Diego_Velazquez      Jan_van_Eyck		Rene_Magritte
Edgar_Degas	     Joan_Miro			Salvador_Dali
Edouard_Manet	     Kazimir_Malevich		Sandro_Botticelli
Edvard_Munch	     Leonardo_da_Vinci		Titian
El_Greco	     Marc_Chagall		Vasiliy_Kandinskiy
Eugene_Delacroix     Michelangelo		Vincent_van_Gogh
Francisco_Goya	     Mikhail_Vrubel		William_Turner


In [ ]:
import os
import cv2

INPUT_DIR = "/content/images/images/"
OUTPUT_DIR = "/content/resized/"
DIMS = (512, 512)

os.mkdir(OUTPUT_DIR)

# Comment the loop below to resize only a specific artist
# Use this to resize all images
for artist in os.listdir(INPUT_DIR):
  os.mkdir(OUTPUT_DIR + artist)
  for name in os.listdir(INPUT_DIR + artist):
    raw = cv2.imread(INPUT_DIR + artist + "/" + name)
    out = cv2.resize(raw, DIMS)
    cv2.imwrite(OUTPUT_DIR + artist + "/" + name, out)


# Uncomment below and comment the above paragraph to resize a specific artist, replace artist with value provided above
# artist = "Vincent_van_Gogh"
# os.mkdir("output/" + artist)
# for name in os.listdir(INPUT_DIR + artist):
#   raw = cv2.imread(INPUT_DIR + artist + "/" + name)
#   out = cv2.resize(raw, dims)
#   cv2.imwrite(OUTPUT_DIR + artist + "/" + name, out)


In [ ]:
ARTIST = "Vincent_van_Gogh"
OUTPUT_DIR = "/content/resized/"
!zip -r -j -q /content/dataset.zip {OUTPUT_DIR + ARTIST + "/"}

# Connecting to Google Drive
This will allow us to save the state of our model and our dataset so we can come back to the Colab as needed.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Move the dataset from temporary to Google Drive memory - this only needs to be done once. Change DATASET_LOC if needed.

In [ ]:
DATASET_LOC = "/content/dataset.zip"
!mkdir /content/drive/MyDrive/MyPaintingGAN
!cp {DATASET_LOC} /content/drive/MyDrive/MyPaintingGAN

# Setting up StyleGAN2 and PyTorch


Only for first run, downloads the repo for StyleGAN2 onto your Google Drive

In [ ]:
%cd /content/drive/MyDrive/MyPaintingGAN
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch

/content/drive/MyDrive/MyPaintingGAN
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 18.22 MiB/s, done.
Resolving deltas: 100% (57/57), done.


Use the below lines to check the documentation for train.py

In [ ]:
%cd /content/drive/MyDrive/MyPaintingGAN/stylegan2-ada-pytorch/

In [ ]:
!python train.py --help

Running StyleGAN2

For the first run, adjust the dataset path if needed, and resume from if you would like to use a different resolution. The available pretrained networks to resume from can be seen by running the command above. You can also train the network without pretrained networks, but that can take very long computation times. Check the official StyleGAN repo for more information on training from scratch.

In [ ]:
%cd /content/drive/MyDrive/MyPaintingGAN/stylegan2-ada-pytorch/

dataset_path = '/content/drive/MyDrive/MyPaintingGAN/dataset.zip'
resume_from = 'ffhq512'
aug_strength = 0.019
train_count = 4
mirror_x = True

gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_count = 4

In [ ]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./output --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

# Continuing the training

Every 4 ticks, which you can see in the log.txt or the log from the cell above, a new snapshot of the network will be saved. This will contain your latest trained data saved as a '.pkl' or pickle file. Colab may randomly shut down the server due to its time constraints, so to train effectively you will need to continue training multiple times.

After the first run, make sure to change resume from to the full path of your 'pkl' file. This will be available under /content/drive/MyDrive/MyPaintingGAN/stylegan2-ada-pytorch/output. Find the latest '.pkl' file and change resume_from to point to it every time you want to continue training. Under the output folder, you will also find sample images from the network. To generate new images use the code below.

# Generating Images from Network

The code below will generate random images and store them in the output folder. To use simply set the location of the latest '.pkl' file trained above, and choose SEED_START and SEED_END. The code will generate a unique image for each seed number. 

Note: if you run the same network with the same seed values you will get the same pictures for each individual seed, to get different pictures, change the seed range.

In [ ]:
SEED_START = 1
SEED_END = 100
NETWORK_LOCATION = "/content/drive/MyDrive/MyPaintingGAN/stylegan2-ada-pytorch/results/yourdirectory/yournetworkfile.pkl"
OUTPUT = "/content/drive/MyDrive/MyPaintingGAN/generated/"


!python generate.py --outdir={OUTPUT} --trunc=0.7 --seeds={SEED_START}-{SEED_END} --network={NETWORK_LOCATION}

# Important Notes

This is a very basic use of StyleGAN2 with limited customization. If you would like to learn more about how it works and how to use StyleGAN efficiently, visit the Github. An important part that is ommited is testing the accuracy of the model. There are many ways in which you can validate a model, but many of them are too complicated to include here. 



Thanks to all the resources made available for this project:

https://github.com/NVlabs/stylegan2-ada-pytorch for the network training repo.

https://www.kaggle.com/ikarus777/best-artworks-of-all-time for compiling artwork to use in this project.

www.artchallenge.ru for providing the artwork compiled.